# CHELSEAS v. BRENTFORD - match analysis
*This notebook intends to walk through the theory and calculations I am currently using in my model to calculate probability of 1 x 2 result and a table of over / under goal projections. I will break the calculations out into utility functions stored in `../src/utils` for ease of use in future explorations. This is to document the calculations.*

### Instructions:

1. Ensure all cells in all notebooks in `/notebooks` directory have been run.
2. Define `home_side` and `away_side` in first cell. If making a comparison with odds from `03_retrieve_odds.ipynb` also enter matchday.
3. Click 'run all'
4. Probability for 1 x 2 and over / under can be viewed in the last cell
5. *Suggestion:* copy and paste notebook to `../output/< matchday_directory >/` and rename file. IMPORTANT: Update file path in lines 14 and 15 if needed if you make a copy.



### Import League Data & Define Sides

Define 'home_side' and 'away_side'
Ensure correct output



In [9]:
import pandas as pd
from scipy.stats import poisson
import numpy as np

"""
BUG: if copying from `notebooks/03_analysis_template_notebook.ipynb` to a directory in output
for storage, ensure the file path for reading the csv will need to be updated from '../data/processed/home_table.csv'
to '../../data/processed/home_table.csv' if frames are run again.

TLDR
if you are in ./notebooks/03_analysis_tempalte.csv you may run this cell as is. If you are not, check that following
read_csv commands read from the correct directory.
"""
home_dataframe = pd.read_csv('../data/processed/home_table.csv')
away_dataframe = pd.read_csv('../data/processed/away_table.csv')

"""
!!! INPUT TEAMS & MATCHDAY HERE !!!

wrapped in quotes, eg.

home_side = "Arsenal"
away_side = "Everton"

If there is a ValueError,
(1) check spelling against the error message. Must be exact.
(2) check capitalization
(3) check that it is wrapped in quotes

!!! INPUT TEAMS & MATCHDAY HERE !!!
"""
home_side = "Chelsea"
away_side = "Brentford"
matchday = "2024-12-15"

# validate input
clubs = set(home_dataframe['Team'].values)
if home_side not in clubs:
    raise ValueError(f"Invalid entry for 'home_side'. Expected one of {clubs}, but received '{home_side}'")
elif away_side not in clubs:
    raise ValueError(f"Invalid entry for 'away_side'. Expected one of {clubs}, but received '{away_side}'")
else:
    print(f"Ready to calculate probability for {home_side} v. {away_side}")

Ready to calculate probability for Chelsea v. Brentford


### Calculate Projected Goals

In [10]:
# 2. grab their gpm_scored and gpm_conceded
home_side_gpm_scored = home_dataframe.loc[
    home_dataframe["Team"] == home_side,
    "gpm_scored"
].values[0]

home_side_gpm_conceded = home_dataframe.loc[
    home_dataframe["Team"] == home_side,
    "gpm_conceded"
].values[0]

print(home_side_gpm_scored, home_side_gpm_conceded)

away_side_gpm_scored = away_dataframe.loc[
    away_dataframe["Team"] == away_side,
    "gpm_scored"
].values[0]

away_side_gpm_conceded = away_dataframe.loc[
    away_dataframe["Team"] == away_side,
    "gpm_conceded"
].values[0]

print(away_side_gpm_scored, away_side_gpm_conceded)

# 3.
epl_home_avg_gpm_scored = round(sum(home_dataframe["gpm_scored"].values) / 20, 2)
epl_home_avg_gpm_conceded = round(sum(home_dataframe["gpm_conceded"].values) / 20, 2)
print(epl_home_avg_gpm_scored, epl_home_avg_gpm_conceded)

# 4. create a ratio (attack and defense rating) of their gpm to the league average gpm
home_attack_rating = home_side_gpm_scored / epl_home_avg_gpm_scored
away_defense_rating = away_side_gpm_conceded / epl_home_avg_gpm_scored
away_attack_rating = away_side_gpm_scored / epl_home_avg_gpm_conceded
home_defense_rating = home_side_gpm_conceded / epl_home_avg_gpm_conceded

print(home_attack_rating, away_defense_rating, away_attack_rating, home_defense_rating)

# home projected goals
home_projected_goals = home_attack_rating * away_defense_rating * epl_home_avg_gpm_scored
# away projected goals
away_projected_goals = away_attack_rating * home_defense_rating * epl_home_avg_gpm_conceded
print("\n ================== projected_goals ==================== \n")
print(f"HOME {home_side} projected goals:", home_projected_goals)
print(f"AWAY {away_side} projected goals: ", away_projected_goals)

1.7142857142857142 1.1428571428571428
0.7142857142857143 2.0
1.63 1.38
1.0517090271691498 1.2269938650306749 0.5175983436853002 0.8281573498964804

 ================== projected_goals ==================== 

HOME Chelsea projected goals: 2.1034180543382996
AWAY Brentford projected goals:  0.5915409642117717


### Poisson Distribution of Goal Probabilities

In [11]:
# Poisson probability for home team goals 0 - 8
home_score_prob = []
away_score_prob = []
for i in range(9):
    prob = poisson.pmf(i, home_projected_goals)
    home_score_prob.append(prob)
for i in range(9):
    prob = poisson.pmf(i, away_projected_goals)
    away_score_prob.append(prob)

home_score_prob = np.array(home_score_prob)
away_score_prob = np.array(away_score_prob)
# print(home_score_prob)
# print(away_score_prob)

exact_score_prob = np.outer(away_score_prob, home_score_prob)
print(f"{home_side} v. {away_side}", exact_score_prob)

Chelsea v. Brentford [[6.75451498e-02 1.42075688e-01 1.49422283e-01 1.04765843e-01
  5.50915913e-02 2.31761295e-02 8.12484822e-03 2.44142178e-03
  6.41916330e-04]
 [3.99557230e-02 8.40435892e-02 8.83894015e-02 6.19732876e-02
  3.25889330e-02 1.37096300e-02 4.80618055e-03 1.44420099e-03
  3.79719805e-04]
 [1.18177235e-02 2.48576129e-02 2.61429759e-02 1.83298692e-02
  9.63884443e-03 4.05490388e-03 1.42152634e-03 4.27152023e-04
  1.12309910e-04]
 [2.33022251e-03 4.90143210e-03 5.15488039e-03 3.61428949e-03
  1.90059044e-03 7.99547250e-04 2.80297020e-04 8.42259733e-05
  2.21453041e-05]
 [3.44605518e-04 7.24849468e-04 7.62330729e-04 5.34500073e-04
  2.81069276e-04 1.18241238e-04 4.14517924e-05 1.24557784e-05
  3.27496364e-06]
 [4.07696561e-05 8.57556306e-05 9.01899709e-05 6.32357377e-05
  3.32527981e-05 1.39889072e-05 4.90408665e-06 1.47362063e-06
  3.87455029e-07]
 [4.01948694e-06 8.45466141e-06 8.89184372e-06 6.23442154e-06
  3.27839871e-06 1.37916861e-06 4.83494691e-07 1.45284495e-07
  3

### Under Market

In [12]:
thresholds = [0.5, 1.5, 2.5, 3.5]
sums = []

for t in thresholds:
    mask = np.add.outer(
        np.arange(
            exact_score_prob.shape[0]
        ),
        np.arange(
            exact_score_prob.shape[1]
        )
    ) <= t
    # print(mask)
    sums.append(exact_score_prob[mask].sum())

# print(sums)
under_market_df = pd.DataFrame({
    "Goals": thresholds,
    "Prob": sums
})
under_market_df["Implied Odds"] = 1 / under_market_df["Prob"]
print("~  ~ under market ~  ~\n", f"{home_side} v. {away_side}\n", under_market_df)

~  ~ under market ~  ~
 Chelsea v. Brentford
    Goals      Prob  Implied Odds
0    0.5  0.067545     14.804912
1    1.5  0.249577      4.006787
2    2.5  0.494860      2.020773
3    3.5  0.715203      1.398204


In [13]:
thresholds = [0.5, 1.5, 2.5, 3.5]
sums = []

for t in thresholds:
    mask = np.add.outer(
        np.arange(
            exact_score_prob.shape[0]
        ),
        np.arange(
            exact_score_prob.shape[1]
        )
    ) >= t
    sums.append(exact_score_prob[mask].sum())

# print(sums)
over_market_df = pd.DataFrame({
    "Goals": thresholds,
    "Prob": sums
})

over_market_df["Implied Odds"] = 1 / over_market_df["Prob"]
print("~  ~ over market ~  ~\n", f"{home_side} v. {away_side}\n", over_market_df)

~  ~ over market ~  ~
 Chelsea v. Brentford
    Goals      Prob  Implied Odds
0    0.5  0.932114      1.072831
1    1.5  0.750082      1.333187
2    2.5  0.504799      1.980988
3    3.5  0.284455      3.515488


### 1 x 2 Probability

In [14]:
# Poisson probability for home team goals 0 - 8
home_score_prob = []
away_score_prob = []
for i in range(9):
    prob = poisson.pmf(i, home_projected_goals)
    home_score_prob.append(prob)
for i in range(9):
    prob = poisson.pmf(i, away_projected_goals)
    away_score_prob.append(prob)

home_score_prob = np.array(home_score_prob)
away_score_prob = np.array(away_score_prob)
# print(home_score_prob)
# print(away_score_prob)

exact_score_prob = np.outer(away_score_prob, home_score_prob)

# print(exact_score_prob)
home_win_prob = np.sum(np.triu(exact_score_prob, k=1))
away_win_prob = np.sum(np.tril(exact_score_prob, k=-1))
draw_prob = np.trace(exact_score_prob)

# prepare data in readable dataframe
data = {
    "winner": [f"{home_side} (1)", "Draw (x)", f"{away_side} (2)"],
    "prob": [home_win_prob, draw_prob, away_win_prob],
}
result_df = pd.DataFrame(data)
result_df['implied_odds'] = 1 / result_df['prob']
print("      ~  ~ moneyline market ~  ~\n", result_df)

      ~  ~ moneyline market ~  ~
           winner      prob  implied_odds
0    Chelsea (1)  0.726285      1.376870
1       Draw (x)  0.181642      5.505348
2  Brentford (2)  0.091732     10.901284


# Full Results Summary:

In [15]:
# helper function to translate discrepency between Understat and Odds-Api naming
def translate(string):
    translations = {
        "Ipswich": "Ipswich Town",
        "Leicester": "Leicester City"
    }
    if string in translations:
        return translations["string"]
    return string

home_side, away_side = translate(home_side), translate(away_side)

match_string = f"{home_side} v. {away_side}"

print(f"CALCULATED PROBABILITIES FOR {home_side} v. {away_side}")
print("\n\n        ~ ~ moneyline market ~ ~\n\n", result_df)
print("\n\n          ~ ~ over market ~ ~ \n\n", over_market_df)
print("\n\n          ~ ~ under market ~ ~ \n\n", under_market_df)

try:
    odds_df = pd.read_csv(f"../data/processed/odds/odds_matchday_{matchday}.csv")
    filtered_odds = odds_df.loc[odds_df["match"] == match_string]
    implied = {
        "1": 1 / filtered_odds["1"],
        "x": 1 / filtered_odds["x"],
        "2": 1 / filtered_odds["2"],
        "point": filtered_odds["ovr_und_point"],
        "over": 1 / filtered_odds["over"],
        "under": 1 / filtered_odds["under"]
    }
    implied_odds_df = pd.DataFrame(implied)

    print(f"\n ~~ MyBookie Odds for {match_string} ~~\n",filtered_odds.drop(columns=["match"]))
    print(f"\n ~~ MyBookie Odds IMPLIED for {match_string} ~~\n", implied_odds_df)
except:
    print(f"No data for sportsbook odds on matchday {matchday}. To view against real odds, run `notebooks/03_retrieve_odds.ipynb` ran correctly and ensure matchday is correct in the first cell of this notebook")


CALCULATED PROBABILITIES FOR Chelsea v. Brentford


        ~ ~ moneyline market ~ ~

           winner      prob  implied_odds
0    Chelsea (1)  0.726285      1.376870
1       Draw (x)  0.181642      5.505348
2  Brentford (2)  0.091732     10.901284


          ~ ~ over market ~ ~ 

    Goals      Prob  Implied Odds
0    0.5  0.932114      1.072831
1    1.5  0.750082      1.333187
2    2.5  0.504799      1.980988
3    3.5  0.284455      3.515488


          ~ ~ under market ~ ~ 

    Goals      Prob  Implied Odds
0    0.5  0.067545     14.804912
1    1.5  0.249577      4.006787
2    2.5  0.494860      2.020773
3    3.5  0.715203      1.398204

 ~~ MyBookie Odds for Chelsea v. Brentford ~~
      1    x    2  ovr_und_point  over  under
1  1.4  5.5  6.6            3.5  1.89   1.92

 ~~ MyBookie Odds IMPLIED for Chelsea v. Brentford ~~
           1         x         2  point      over     under
1  0.714286  0.181818  0.151515    3.5  0.529101  0.520833


# Conclusion and Pick

SAFE PICK: Chelsea is relatively reliable this season and are valued how we expect. *WAFPEL?*

### My Pick:

**Chelsea @ 1.4** -- *1 unit*